In [ ]:
import pandas as pd
import os 
import re 
import numpy as np 
import string 
import tabula


directory = 'path to PDF docs' #Cesta k PDF souborům

In [ ]:
text_zpravy = {}

#Extrakce textu z PDF souboru do text_zpravy slovniku za pomoci tabula-read_pdf s atributem Stream
for filename in os.listdir(directory):
    print("zpracovavam zpravu: ", filename)
    if filename.endswith('.pdf'):
        filepath = os.path.join(directory, filename)
        tables = tabula.read_pdf(filepath, pages='all', stream=True, silent=True)
        page_tables = []
        for table in tables:
            df = table.copy()
            page_tables.append(df)
        text_zpravy[filename] = page_tables

In [ ]:
text_zpravy_filtered = {}

# Filtruje pouze ty tabulky, v kterých je název cílového sloupce, případně ty, kde je název cílového sloupce v prvním řádku a posune ho do názvu sloupce
for filename, tables in text_zpravy.items():
    page_tables = []
    for table in tables:
        # Kontroluje, zda se nějaký sloupec shoduje s jedním z požadovaných názvů.
        columns = []
        has_required_columns = False
        for col in table.columns:
            if 'Běžné období' in col or '31.prosince 2021' in col or '31. prosince 2021' in col or '31.12.2021' in col or '31. 12. 2021' in col or '2021' in col or '(tis. Kč) 2021' in col:
                if isinstance(table[col][0], str):
                    columns.append(table[col][0])
                else:
                    columns.append(col)
                has_required_columns = True
            else:
                columns.append(col)
        # Kontroluje, zda se nějaký sloupec vyskytuje v tabulce
        if has_required_columns or any(col for col in columns if 'Běžné období' in col or '31.prosince 2021' in col or '31. prosince 2021' in col or '31.12.2021' in col or '31. 12. 2021' in col or '2021' in col or '(tis. Kč) 2021' in col):
            df = table.copy()
            if not any(col for col in df.columns if 'Běžné období' in col or '31.prosince 2021' in col or '31. prosince 2021' in col or '31.12.2021' in col or '31. 12. 2021' in col or '2021' in col or '(tis. Kč) 2021' in col):
                df.columns = columns[0:len(df.columns)]
            page_tables.append(df)
    # Přidá list tabulek do slovníku, kde jméno klíče je jméno souboru
    if page_tables:
        text_zpravy_filtered[filename] = page_tables


In [ ]:
selected_tables = {}

#Vybere pouze tabulky, kde je řádek s názvem Pohledávky za bankami.
for filename, tables in text_zpravy_filtered.items():
    for table in tables:
        if any(row.str.contains('Pohledávky za bankami').any() for _, row in table.iterrows()):
            if filename not in selected_tables:
                selected_tables[filename] = []
            selected_tables[filename].append(table)


In [ ]:
selected_tables_pasiva = {}

#Vybere pouze tabulky, kde je řádek s názvem Ostatní pasiva.
for filename, tables in text_zpravy_filtered.items():
    for table in tables:
        if any(row.str.contains('Ostatní pasiva').any() for _, row in table.iterrows()):
            if filename not in selected_tables_pasiva:
                selected_tables_pasiva[filename] = []
            selected_tables_pasiva[filename].append(table)

In [ ]:
first_selected_table = {}

#Vybere první tabulku z tabulek selected_tables
#Musíme dávat pozor, jestli první tabulka je ta co chceme.
for filename, tables in selected_tables.items():
    if len(tables) > 0:
        first_selected_table[filename] = tables[0]

In [ ]:
first_selected_table_pasiva = {}

#Vybere první tabulku z tabulek selected_tables_pasiva
#Musíme dávat pozor, jestli první tabulka je ta co chceme.
for filename, tables in selected_tables_pasiva.items():
    if len(tables) > 0:
        first_selected_table_pasiva[filename] = tables[0]

In [ ]:
#Vytváříme finální slovník s výsledky. Požadované hodnoty jsou ve formátu float
selected_values_first_table = {}
#Zde již vybíráme konkrétní hodnoty na základě názvu sloupce i řádků
for filename, table in first_selected_table.items():
    try:
        # Extrahuje požadované hodnoty z tabulky
        selected_columns = [col for col in table.columns if 'Běžné období' in col or '31.prosince 2021' in col or '31. prosince 2021' in col or '31.12.2021' in col or '31. 12. 2021' in col or '2021' in col or '(tis. Kč) 2021' in col]
        selected_rows_1 = [row for _, row in table.iterrows() if row.str.contains('Pohledávky za bankami').any()]
        selected_rows_2 = [row for _, row in table.iterrows() if row.str.contains('Dluhové cenné papíry').any()]
        selected_rows_3 = [row for _, row in table.iterrows() if row.str.contains('Aktiva celkem|AKTIVA CELKEM').any()]
        selected_rows_4 = [row for _, row in table.iterrows() if row.str.contains('Ostatní pasiva').any()]
        selected_rows_5 = [row for _, row in table.iterrows() if row.str.contains('Zisk za účetní období|Zisk/ztráta za účetní období|Zisk \(\+\) / ztráta \(-\) za účetní období|Zisk nebo ztráta za účetní období').any()]
        table_for_selection = table

        # Tento blok probíhá v případě, že jsou tabulky pasiva a aktiva oddělené, tzn. pasiva nebyla nalezena v první tabulce
        if not selected_rows_4 or not selected_rows_5:
            # Pokud nastane, tak hledáme tabulku z first_selected_table_pasiva
            table_for_selection = first_selected_table_pasiva[filename]
            selected_rows_4 = [row for _, row in table_for_selection.iterrows() if row.str.contains('Ostatní pasiva').any()]
            selected_rows_5 = [row for _, row in table_for_selection.iterrows() if row.str.contains('Zisk za účetní období|Zisk/ztráta za účetní období|Zisk \(\+\) / ztráta \(-\) za účetní období|Zisk nebo ztráta za účetní období').any()]
            # Vytváříme nový slovník values
            values = {}
            # Iterujem pro řádky 1-3 z first_selected_table
            for row in selected_rows_1 + selected_rows_2 + selected_rows_3:
                row_name = row.values[0]
                for col in selected_columns:
                    if isinstance(table.at[row.name, col], str) and table.at[row.name, col].strip() != '':
                        value_str = table.at[row.name, col].strip()
                        try:
                            value = float(table.at[row.name, col].replace(' ', '').replace(',', '.'))
                            values[row_name] = value
                            break
                        except ValueError:
                            match = re.search(r'\d+(\.\d+)?', value_str)
                            if match:
                                value = float(match.group())
                                values[row_name] = value
                                break
            # Iterujem pro řádky 4-5 z tabulky first_selected_table_pasiva
            for row in selected_rows_4 + selected_rows_5:
                row_name = row.values[0]
                for col in selected_columns:
                    if isinstance(table_for_selection.at[row.name, col], str) and table_for_selection.at[row.name, col].strip() != '':
                        value = float(table_for_selection.at[row.name, col].replace(' ', '').replace(',', '.'))
                        values[row_name] = value
                        break
        else:
            table_for_selection = table
            # Opět vytváříme nový slovník values
            values = {}
            # Iterujem rovnou přes všechny řádky z tabulky first_selected_table
            for row in selected_rows_1 + selected_rows_2 + selected_rows_3 + selected_rows_4 + selected_rows_5:
                row_name = row.values[0]
                for col in selected_columns:
                    if isinstance(table_for_selection.at[row.name, col], str) and table_for_selection.at[row.name, col].strip() != '':
                        value = float(table_for_selection.at[row.name, col].replace(' ', '').replace(',', '.'))
                        values[row_name] = value

        #Finální přiřazení do cílové proměnné    
        selected_values_first_table[filename] = values
    except Exception as e:
        print(f"Chyba nastala u dokumentu: {filename}. Chyba je: {e}  \n Pokračuji dále...")
        continue

In [ ]:
selected_values_first_table.keys()

In [ ]:
selected_values_first_table